## train VGG model with PAC-Bayes information bottleneck.

In [1]:
import numpy as np
import torch
import os
import pdb

from src.dataset import load_data
from src.utils import img_preprocess, setup_seed, predict, eval_metric, feature_map_size
from src.utils import train
from src.models import VGG
from src.pib_utils import train_pib

In [2]:
__data_set__ = 'cifar10'

__prior_ckpt__ = './checkpoints/{}/vgg_prior.pt'.format(__data_set__)
__save_ckpt__ = './checkpoints/{}/vgg_pib.pt'.format(__data_set__)

opt = {
    'num_epoch':100,
    'batch_size':32,
    'lr':1e-4, 
    'weight_decay':0,
    'beta':1e-1,
    'noise_scale':1e-10,
    'schedule': [50, 80],
    'early_stop': 10,
}
if not os.path.exists('./checkpoints/{}'.format(__data_set__)):
    os.makedirs('./checkpoints/{}'.format(__data_set__))

# set random seed
setup_seed(2020)

## load data & preprocess

In [3]:
x_tr, y_tr, x_va, y_va, x_te, y_te = load_data(__data_set__)

all_tr_idx = np.arange(len(x_tr))
num_class = np.unique(y_va).shape[0]

x_tr, y_tr = img_preprocess(x_tr, y_tr,)
x_va, y_va = img_preprocess(x_va, y_va,)
x_te, y_te = img_preprocess(x_te, y_te,)

load from CIFAR-10.


## train PIB-based VGG model

In [4]:
# load model
model = VGG(num_classes=num_class, dropout_rate=0.0, last_feature_map_size=feature_map_size(__data_set__))
model.cuda()

VGG(
  (extract_feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_feature

In [5]:
# get prior on the validation set
if os.path.exists(__prior_ckpt__):
    print("load prior.")
    model.load_state_dict(torch.load(__prior_ckpt__))
else:
    train(model, np.arange(len(y_va)), x_va, y_va, x_va, y_va, 10, 32, 5e-5, 0, __prior_ckpt__, 5)
w0_dict = dict()
for param in model.named_parameters():
    w0_dict[param[0]] = param[1].clone().detach() # detach but still on gpu
model.w0_dict = w0_dict
model._initialize_weights()
print("done get prior weights")

load prior.
initialize model weights.
done get prior weights


In [6]:
# start training model
info_dict, loss_acc_dict = train_pib(model, all_tr_idx,
    x_tr, y_tr, x_va, y_va, 
    num_epoch=opt['num_epoch'],
    batch_size=opt['batch_size'],
    lr=opt['lr'],
    weight_decay=opt['weight_decay'],
    beta=opt['beta'],
    early_stop_ckpt_path=__save_ckpt__,
    early_stop_tolerance=opt['early_stop'],
    noise_scale=opt['noise_scale'],
    schedule=opt['schedule'],
    )

  0%|                                                                                                  | 0/100 [00:00<?, ?it/s]D:\python\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
D:\PAC-Bayes IB\PAC-Bayes-IB\src\pib_utils.py:115: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1025.)
  p.data.add_(-group['lr'], d_p)


epoch: 0, info: {'extract_feature.0.weight': tensor(3.6269e-06, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.2.weight': tensor(2.5803e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.5.weight': tensor(9.1582e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.7.weight': tensor(5.9072e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.10.weight': tensor(7.7142e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.12.weight': tensor(7.5887e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.14.weight': tensor(9.1890e-07, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.0.weight': tensor(1.3563e-06, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.3.weight': tensor(9.2734e-07, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.6.weight': tensor(9.8919e-07, device='cuda:0', grad_fn=<PowBackward0>)}
epoch: 0, tr loss: 0.027935729106155428, lr: 0.0001, e_decay: 1.1113430446130224e-05


  1%|▉                                                                                         | 1/100 [00:18<29:47, 18.06s/it]

epoch: 0, va acc: 0.8075999617576599
epoch: 1, info: {'extract_feature.0.weight': tensor(1.0749e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.2.weight': tensor(2.6885e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.5.weight': tensor(6.0926e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.7.weight': tensor(6.9039e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.10.weight': tensor(4.3970e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.12.weight': tensor(5.7491e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.14.weight': tensor(8.2988e-07, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.0.weight': tensor(8.3694e-07, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.3.weight': tensor(6.8296e-07, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.6.weight': tensor(8.9229e-07, device='cuda:0', grad_fn=<PowBackward0>)}
epoch: 1, tr loss: 0.016418349326268174, lr: 0.0001, e_decay: 5.9326

  2%|█▊                                                                                        | 2/100 [00:36<29:35, 18.12s/it]

...

epoch: 16, va acc: 0.8057999610900879
epoch: 17, info: {'extract_feature.0.weight': tensor(1.1708e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.2.weight': tensor(3.3227e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.5.weight': tensor(3.2608e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.7.weight': tensor(2.8221e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.10.weight': tensor(2.4811e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.12.weight': tensor(2.0055e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.14.weight': tensor(2.9486e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.0.weight': tensor(4.0754e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.3.weight': tensor(2.6790e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.6.weight': tensor(3.0805e-08, device='cuda:0', grad_fn=<PowBackward0>)}
epoch: 17, tr loss: 0.0012668993128090184, lr: 0.0001, e_decay: 3.

 18%|████████████████                                                                         | 18/100 [05:24<24:37, 18.02s/it]

epoch: 17, va acc: 0.8057999610900879
epoch: 18, info: {'extract_feature.0.weight': tensor(1.8303e-07, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.2.weight': tensor(7.4612e-09, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.5.weight': tensor(1.5474e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.7.weight': tensor(1.7633e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.10.weight': tensor(2.4692e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.12.weight': tensor(2.2989e-08, device='cuda:0', grad_fn=<PowBackward0>), 'extract_feature.14.weight': tensor(2.0370e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.0.weight': tensor(5.2487e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.3.weight': tensor(2.9942e-08, device='cuda:0', grad_fn=<PowBackward0>), 'classifier.6.weight': tensor(2.6500e-08, device='cuda:0', grad_fn=<PowBackward0>)}
epoch: 18, tr loss: 0.0011806567544521713, lr: 0.0001, e_decay: 4.

 18%|████████████████                                                                         | 18/100 [05:42<25:58, 19.00s/it]

epoch: 18, va acc: 0.8068000078201294
early stop on epoch 18, val acc 0.8087999820709229


## bootstrap on test set

In [7]:
# do bootstrapping
stats = []
for i in range(10):
    # sample x_te
    sub_idx = np.random.choice(np.arange(len(x_te)), len(x_te), replace=True)
    x_te_sub, y_te_sub = x_te[sub_idx], y_te[sub_idx]
    pred_te = predict(model, x_te_sub)
    acc_te = eval_metric(pred_te, y_te_sub, num_class)
    stats.append(acc_te)
    print("test acc:", acc_te)

# compute confidence interveal 95%
alpha = 0.95
p = ((1-alpha)/2) * 100
lower = max(0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))
print('%.1f confidence interval %.2f%% and %.2f%%' % (alpha*100, lower*100, upper*100))
print('average:', (upper+lower)/2)
print('interval:', (upper-lower)/2)

test acc: tensor(0.8002, device='cuda:0')
test acc: tensor(0.8009, device='cuda:0')
test acc: tensor(0.8128, device='cuda:0')
test acc: tensor(0.8011, device='cuda:0')
test acc: tensor(0.8118, device='cuda:0')
test acc: tensor(0.8077, device='cuda:0')
test acc: tensor(0.8032, device='cuda:0')
test acc: tensor(0.7972, device='cuda:0')
test acc: tensor(0.8063, device='cuda:0')
test acc: tensor(0.8019, device='cuda:0')
95.0 confidence interval 79.79% and 81.26%
average: tensor(0.8052, device='cuda:0')
interval: tensor(0.0073, device='cuda:0')
